In [1]:
import pandas as pd
import re
import string
string.punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [2]:
lyrics = "[Verse 1]Please save a kid that needs some helpAnd I can’t begin to think of anything that could helpBut the weed is guaranteed, indeed, just what I needHow I feel, upon a time, so recent in time, made me sadWhen I recollect how it used to beLike David and Goliath, kinda like meAnd the Devil tryna rip out my soulTryna catch a nigga on sleep, no-noYou can try again and I’ll be readyWon’t let you kill me in my dream like Freddy KruegerNo, I’m not no loser, I’ll see you in Hell[Chorus]At the end of the day, dayMy mama told me don’t let no one break me, let no one break me(Yeah, yeah, yeah, no)At the end of the day, dayNobody, nobody ever could stop me, ever could stop me(Yeah, yeah, no)At the end of the day, dayYou can’t regret it if you were trying, if you were trying(Yeah, yeah, no)At the end of the day, dayI’m walking with a heart of a lion, yeah, [Verse 2]Please save a kid that needs some helpAnd I can’t see ahead of me so I move in stealthHide and seek within a dream I seem to glide above my horrorThough I feel I’ll never be complete inside the dark I borrowTo proceed and remain intact my mental is so unstableAnd they talk and judge a manThey have no clue of what I’m capable’Til I show a side of me no one had thought could be withinI told you, no I’m not a loser, I’ll see you in Hell[Chorus]At the end of the day, dayMy mama told me don’t let no one break me, let no one break me(Yeah, yeah, yeah, no)At the end of the day, dayNobody, nobody ever could stop me, ever could stop me(Yeah, yeah, no)At the end of the day, dayYou can’t regret it if you were trying, if you were trying(Yeah, yeah, no)At the end of the day, dayI’m walking with a heart of a lion, yeah[Bridge]No, no, no, no, no, no, noThe heart of a lion, the heart of a lionYeah, no, yeah (Yeah)No, no, no, no, no, no, noThe heart of a lionYeah, no, yeah (Yeah)No, no, no, no, no, no, noThe heart of a lion, the heart of a lionYeah, no, yeah (Yeah)No, no, no, no, no, no, noYeah, no, yeah (Yeah)[Verse 3]I’m just a kid who needs no helpI’ve achieved a bigger thingNo cash or check could be more blessedAnd if you had ever seen the things I’ve seen up in my slumberYou’d be shook, stay awakeNo need to run, your fate would followYou should know, not for the faint at heartThis world understands no cowardIt’s a goal, a simple code, I stay on the grind to the 25th hourAnd I know the fight in me is somewhere hiding deep withinI told you, no, I’ll never let you drag me down to Hell[Chorus]At the end of the day, dayMy mama told me don’t let no one break me, let no one break me(Yeah, yeah, yeah, no)At the end of the day, dayNobody, nobody ever could stop me, ever could stop me(Yeah, yeah, no)At the end of the day, dayYou can’t regret it if you were trying, if you were tryingAt the end of the day, day(Yeah, yeah, no)I’m walking with a heart of a lion, yeah, ‘[Outro]No, no, no, no, no, no, noYeah, no, yeah (Yeah)No, no, no, no, no, no, noYeah, no, yeah (Yeah)No, no, no, no, no, no, noYeah, no, yeah (Yeah)No, no, no, no, no, no, noYeah, no, yeah (Yeah) Yeah, yeah, yeah...’]"

In [19]:
df = pd.read_csv('../raw_data/rap_data.csv')

In [20]:
df

,Unnamed: 0,Artists,Songs,Year
0,0,ARTIST,TITLE,YEAR
1,1,Birdman,100 Million,2008
2,2,Yung Joc Featuring Marques Houston & Trey Songz,1st Time,2007
3,3,DJ Unk,2 Step,2007
4,4,50 Cent,21 Questions,2003
...,...,...,...,...
316,316,Young Buck Featuring Latoya Williams,You Ain't Goin Nowhere,2007
317,317,Ice Cube Featuring Mack 10 & Ms. Toi,You Can Do It (Put Your Back Into It),2002
318,318,Ice Cube Featuring Mack 10 & Ms. Toi,You Can Do It [Censored Mix],2000
319,319,Guerilla Black Featuring Mario Winans,You're the One,2005


In [48]:
text = df['Songs'].astype('str')
text

0                                      TITLE
1                                100 Million
2                                   1st Time
3                                     2 Step
4                               21 Questions
                       ...                  
316                   You Ain't Goin Nowhere
317    You Can Do It (Put Your Back Into It)
318             You Can Do It [Censored Mix]
319                           You're the One
320                                     Zoom
Name: Songs, Length: 321, dtype: object

In [49]:
def clean_text(text):

    #remove headers like [Chorus] etc
    headers = text.str.findall(r"\[(.*?)\]")
    for header in headers:
        text = text.replace(header, ' ')
    print(text)

        
    #separate lower/upper case words (like 'needHow')
    text = text.str.findall('[A-Z][^A-Z]*')
    new_text = ' '.join(text)
    print(new_text)
    
    #remove punctuation
    exclude = string.punctuation + "’‘”“"
    for punctuation in exclude:
           text = text.replace(punctuation, '')

    #turn text into lowercase
    text = text.lower()

    #remove numericals
    text = ''.join(word for word in text if not word.isdigit())

    #remove stopwords
    stop_words = set(stopwords.words('english'))

    #tokenise
    word_tokens = word_tokenize(text)
    text = [w for w in word_tokens if not w in stop_words]

    #lemmatise
    lemmatizer = WordNetLemmatizer()
    lemmatized = [lemmatizer.lemmatize(word) for word in text]
    text = lemmatized

    text = ' '.join(text)

    #rejoin "wan na" to "wanna"
    wannas = text.str.findall(r"wan na")
    for wanna in wannas:
        text = text.replace(wanna, "wanna")

    return text

In [50]:
test_text = clean_text(text)

0                                      TITLE
1                                100 Million
2                                   1st Time
3                                     2 Step
4                               21 Questions
                       ...                  
316                   You Ain't Goin Nowhere
317    You Can Do It (Put Your Back Into It)
318             You Can Do It [Censored Mix]
319                           You're the One
320                                     Zoom
Name: Songs, Length: 321, dtype: object


TypeError: sequence item 0: expected str instance, list found

In [51]:
telefone = '100 Million'
telefone
#telefone_str = ' '.join(telefone)

'100 Million'

In [4]:
text_list = clean_text(lyrics).split()

In [5]:
text_list

['please',
 'save',
 'kid',
 'need',
 'help',
 'cant',
 'begin',
 'think',
 'anything',
 'could',
 'help',
 'weed',
 'guaranteed',
 'indeed',
 'need',
 'feel',
 'upon',
 'time',
 'recent',
 'time',
 'made',
 'sad',
 'recollect',
 'used',
 'like',
 'david',
 'goliath',
 'kinda',
 'like',
 'devil',
 'tryna',
 'rip',
 'soul',
 'tryna',
 'catch',
 'nigga',
 'sleep',
 'nono',
 'try',
 'ill',
 'ready',
 'wont',
 'let',
 'kill',
 'dream',
 'like',
 'freddy',
 'krueger',
 'im',
 'loser',
 'ill',
 'see',
 'hell',
 'end',
 'day',
 'day',
 'mama',
 'told',
 'dont',
 'let',
 'one',
 'break',
 'let',
 'one',
 'break',
 'yeah',
 'yeah',
 'yeah',
 'end',
 'day',
 'day',
 'nobody',
 'nobody',
 'ever',
 'could',
 'stop',
 'ever',
 'could',
 'stop',
 'yeah',
 'yeah',
 'end',
 'day',
 'day',
 'cant',
 'regret',
 'trying',
 'trying',
 'yeah',
 'yeah',
 'end',
 'day',
 'day',
 'im',
 'walking',
 'heart',
 'lion',
 'yeah',
 'please',
 'save',
 'kid',
 'need',
 'help',
 'cant',
 'see',
 'ahead',
 'move',
 's

In [6]:
data = pd.DataFrame(text_list, columns = ['lyrics'])
data

,lyrics
0,please
1,save
2,kid
3,need
4,help
...,...
321,yeah
322,yeah
323,yeah
324,yeah


In [7]:
vectorizer = TfidfVectorizer().fit(data['lyrics'])
data_vectorized = vectorizer.transform(data['lyrics'])

lda_model = LatentDirichletAllocation(n_components=2).fit(data_vectorized)

In [14]:
def print_topics(mdl, vct):
    for idx, topic in enumerate(mdl.components_):
        print("Topic %d:" % (idx))
        print([(vct.get_feature_names_out()[i], topic[i])
                        for i in topic.argsort()[:-10 - 1:-1]])
        

print_topics(lda_model, vectorizer)

Topic 0:
[('yeah', 51.4724788910277), ('day', 24.47236746509876), ('end', 12.472158262254936), ('could', 9.47201776861004), ('let', 8.471947983388272), ('lion', 8.471947850973345), ('im', 7.471856748662128), ('stop', 6.471735571502849), ('cant', 5.4715731183292755), ('need', 5.471571453700271)]
Topic 1:
[('heart', 9.483634483337669), ('ever', 7.483585795338213), ('one', 7.483585791333568), ('nobody', 6.483549924924741), ('trying', 6.483549900977606), ('break', 6.483549751899127), ('told', 5.483500651556457), ('ill', 5.483500599909846), ('help', 4.4834287664302215), ('like', 3.483317364730529)]


In [9]:
example = ["yeah"]

example_vectorized = vectorizer.transform(example)

lda_vectors = lda_model.transform(example_vectorized)

print("topic 0 :", lda_vectors[0][0])
print("topic 1 :", lda_vectors[0][1])

topic 0 : 0.7497301717141671
topic 1 : 0.2502698282858328


In [10]:
lda_vectors

array([[0.74973017, 0.25026983]])